In [2]:
import pickle
import numpy as np
from keras import backend as K
from matplotlib import pyplot
from sklearn.model_selection import KFold
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import tensorflow as tf


participantCount = 1

rawData = [{}]*participantCount

for i in range(participantCount):
    rawData[i] = pickle.load(open(f'../data/s{i+1:02}.dat', 'rb'), encoding="latin1")

labels = np.array(list(map(lambda participant : participant['labels'], rawData)))
data = np.array(list(map(lambda participant : participant['data'], rawData)))

def get_y(emotion):
    return (labels.reshape(-1, 4)[:,emotion] >= 5).astype(int)

def get_x():
    return data.reshape(-1, 40, 8064)

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [5]:
physical_devices = tf.config.list_physical_devices('CPU')
# tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

Y = get_y(0)
X = np.moveaxis(get_x(), 1, 2)

for i in [0, 1, 3]:
    print("EMOTION NUMBER", i)
    Y = get_y(i)
    kfold = KFold(n_splits=5, shuffle=False)
    cvscores = []
    fscores = []
    for train, test in kfold.split(X, Y):
        model = Sequential()
        model.add(LSTM(80, input_dim=40))
        model.add(Dense(1, activation='sigmoid'))
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=["acc", f1_m, precision_m, recall_m])

        model.fit(X[train], Y[train], validation_data=(X[test], Y[test]), epochs=100, batch_size=128, verbose=2, shuffle=False)

        scores = model.evaluate(X[test], Y[test], verbose=0)
        print("%s: %.2f%%, %s: %.2f%%" % (model.metrics_names[1], scores[1]*100, model.metrics_names[2], scores[2]*100))
        cvscores.append(scores[1] * 100)
        fscores.append(scores[2] * 100)

    print("%.2f%% (+/- %.2f%%), %.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores), np.mean(fscores), np.std(fscores)))    
    print("EMOTION", i)

EMOTION NUMBER 0
Epoch 1/100
1/1 - 1s - loss: 0.8285 - acc: 0.4375 - f1_m: 0.4706 - precision_m: 0.3636 - recall_m: 0.6667 - val_loss: 0.5593 - val_acc: 0.7500 - val_f1_m: 0.8333 - val_precision_m: 1.0000 - val_recall_m: 0.7143
Epoch 2/100
1/1 - 1s - loss: 0.8147 - acc: 0.5000 - f1_m: 0.5000 - precision_m: 0.4000 - recall_m: 0.6667 - val_loss: 0.5690 - val_acc: 0.7500 - val_f1_m: 0.8333 - val_precision_m: 1.0000 - val_recall_m: 0.7143
Epoch 3/100


KeyboardInterrupt: 